In [1]:
# import required packages

import networkx as nx
import numpy as np
import scipy as sp
import pandas as pd

import matplotlib.pyplot as plt
from pyvis.network import Network
net = Network()

import gravis as gv
import csv

## Read in Edgelist

In [2]:
# Read in network edgelist

yeast_file = "yeast_network.txt"

G_exp = nx.read_edgelist(yeast_file,comments="#",nodetype=str)

## Read in CSV

In [3]:
clusters = []

csv_reader = csv.reader(open("dpclus_complexes.csv"))

for cluster in csv_reader:
    clusters.append(cluster)
cluster_rep = []
for cluster in clusters:
    cluster_rep.append(cluster[0])

In [4]:
def find_clusters(clusters, node):
    unknown_clusters = []
    for cluster in clusters:
        if node in cluster:
            unknown_clusters.append(cluster)
     
    return unknown_clusters

## Create Cluster Graph

In [5]:
# Define edge weights based on formula

def cluster_weight(cluster1, cluster2, G):
    # Number of connected pairs / number of pairs in total
    total_num = len(cluster1) * len(cluster2)
    connected_num = 0.0
    for node1 in cluster1:
        for node2 in cluster2:
            if (node1, node2) in G.edges:
                connected_num += 1
    return connected_num / total_num


In [6]:
cluster_weight(clusters[0], clusters[3], G_exp)

0.0244173140954495

In [7]:
cluster_graph = nx.Graph()

In [8]:
cluster_weighted_edges = []

# Assume clusters are sorted
for i in range(len(clusters)):
    for j in range(i+1, len(clusters)):
        cluster_weighted_edges.append((i,j, cluster_weight(clusters[i], clusters[j], G_exp)))

In [9]:
self_loops = [entry for entry in cluster_weighted_edges if entry[0] == entry[1]]

In [10]:
cluster_graph.add_weighted_edges_from(cluster_weighted_edges)

In [11]:
node_size = []
for i in range(0, len(clusters)):
    node_size.append(len(clusters[i]))

 ## PPI Subgraph PyVis Visualisation

In [12]:
PPI_subgraph = Network(notebook = True, bgcolor="#222222", font_color="white")

# set the physics layout of the network
PPI_subgraph.barnes_hut(gravity = -4000, overlap = 1, damping = 0.2)
#print(cluster_rep)
nodes = []
for i in range(0, len(clusters)):
    l = [i, node_size[i], clusters[i]]
    nodes.append(l)
#print(nodes)
for n in nodes:
    PPI_subgraph.add_node(n[0], size = 2*n[1], title = n[2])
    #print(n[0], n[1], n[2])
for e in cluster_weighted_edges:
                src = e[0]
                dst = e[1]
                w = round(200*e[2], 0)
                #print(src, dst, w)
                #PPI_subgraph.add_node(src, src, title=src)
                #PPI_subgraph.add_node(dst, dst, title=dst)
                if w > 50:
                    PPI_subgraph.add_edge(src, dst, value=w, title = w)
                    
neighbor_map = PPI_subgraph.get_adj_list()
for node in [67, 164, 70, 73, 81, 26, 61]:
    PPI_subgraph.get_node(node)['color'] = 'Yellow'
    PPI_subgraph.get_node(node)['weight'] = 100

# add neighbor data to node hover data
# for node in PPI_subgraph.nodes:
                #node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
                #node["value"] = len(neighbor_map[node["id"]])

PPI_subgraph.show("PPIsubgraph.html") # Show PPI Subgraph

PPIsubgraph.html
